# Penetapan tipe data

### Import Library

In [1]:
import numpy as np 
import pandas as pd 
import time

import warnings
warnings.filterwarnings('ignore')

# menampilkan lebih banyak kolom dan baris pada Dataset
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# fungsi rekomendasi tipe data
def rekomendasi_tipe_data(kolom):
    mn = kolom.min()
    mx = kolom.max()
    dtype_lama = kolom.dtype
    if mn >= 0:
        if mx <= 255:
            return 'uint8', dtype_lama
        elif mx <= 65535:
            return 'uint16', dtype_lama
        elif mx <= 4294967295:
            return 'uint32', dtype_lama
        elif mx <= 18446744073709551615:
            return 'uint64', dtype_lama
        else:
            return 'float64', dtype_lama
    else:
        if mn >= -128 and mx <= 127:
            return 'int8', dtype_lama
        elif mn >= -32768 and mx <= 32767:
            return 'int16', dtype_lama
        elif mn >= -2147483648 and mx <= 2147483647:
            return 'int32', dtype_lama
        elif mn >= -9223372036854775808 and mx <= 9223372036854775807:
            return 'int64', dtype_lama
        else:
            return 'float64', dtype_lama
    
def rekomendasi_objek_dtype(kolom):
    unique_values = kolom.unique()
    if len(unique_values) / len(kolom) < 0.5:
        return pd.Categorical(kolom).as_ordered().dtype, kolom.dtype
    else:
        return 'object', kolom.dtype

### Load dataset

In [3]:
# Load Dataset CIC-IDS2018 #
# ======================== #

m_load = time.time()
#dataset_cic2018 = pd.read_csv("G:\\Universitas Indonesia\\Rancangan Paper\\Dataset CSE-CIC-IDS2018-AWS\\CIC2018\\CSE_CIC_IDS2018_Praproses.csv")
#dataset_cic2018 = pd.read_csv("G:\\Universitas Indonesia\\Rancangan Paper\\Dataset CSE-CIC-IDS2018-AWS\\CIC2018\\CIC2018.csv")
dataset_cic2018 = pd.read_csv("D:\\Fando\\Universitas Indonesia\\Semester 4\\Thesis\\Dataset CSE-CIC-IDS2018-AWS\\CIC2018.csv")

s_load = time.time()
print ('Lama waktu memuat dataset : ', s_load - m_load)

Lama waktu memuat dataset :  144.04777121543884


In [4]:
# total penggunaan memori pada tipe data lama
dataset_cic2018.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16232943 entries, 0 to 16232942
Data columns (total 84 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           int64  
 1   Protocol           int64  
 2   Timestamp          object 
 3   Flow Duration      int64  
 4   Tot Fwd Pkts       int64  
 5   Tot Bwd Pkts       int64  
 6   TotLen Fwd Pkts    int64  
 7   TotLen Bwd Pkts    int64  
 8   Fwd Pkt Len Max    int64  
 9   Fwd Pkt Len Min    int64  
 10  Fwd Pkt Len Mean   float64
 11  Fwd Pkt Len Std    float64
 12  Bwd Pkt Len Max    int64  
 13  Bwd Pkt Len Min    int64  
 14  Bwd Pkt Len Mean   float64
 15  Bwd Pkt Len Std    float64
 16  Flow Byts/s        float64
 17  Flow Pkts/s        float64
 18  Flow IAT Mean      float64
 19  Flow IAT Std       float64
 20  Flow IAT Max       int64  
 21  Flow IAT Min       int64  
 22  Fwd IAT Tot        int64  
 23  Fwd IAT Mean       float64
 24  Fwd IAT Std        float64
 25  Fwd IAT Max     

In [5]:
# kolom/fitur yang memiliki NaN >50%
data_2_kolom = ['Timestamp', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP']

# data 1 tanpa NaN
data_1 = dataset_cic2018.drop(columns=data_2_kolom)

# data 2
data_2 = dataset_cic2018[data_2_kolom]

In [6]:
memori_ori = data_1.memory_usage(deep=True) / 1000**2
hasil = pd.DataFrame({'Penggunaan memori lama (MB)': memori_ori})

# rekomendasi tipe data
tipe_data={}
for kolom in data_1.columns:
    if data_1[kolom].dtype == 'object':
        dtype, dtype_lama = rekomendasi_objek_dtype(data_1[kolom])
    else:
        dtype, dtype_lama = rekomendasi_tipe_data(data_1[kolom])
    tipe_data[kolom] = dtype
    
    # konversi kolom ke tipe data yang sesuai
    data_1[kolom] = data_1[kolom].astype(dtype)

    # hitung penggunaan memori setelah konversi
    memori_baru = data_1[kolom].memory_usage(deep=True) / 1000**2

    # Calculate memory reduction
    reduksi = memori_ori[kolom] - memori_baru

    # Add column information to result dataframe
    hasil.loc[kolom, 'Penggunaan memori baru (MB)'] = memori_baru
    hasil.loc[kolom, 'Tipe Data Baru'] = dtype
    hasil.loc[kolom, 'Tipe Data Lama'] = dtype_lama
    hasil.loc[kolom, 'Min'] = data_1[kolom].min()
    hasil.loc[kolom, 'Max'] = data_1[kolom].max()
    hasil.loc[kolom, 'Pengurangan Memori (MB)'] = reduksi
    
# print tipe data baru
print(tipe_data)

{'Dst Port': 'uint16', 'Protocol': 'uint8', 'Flow Duration': 'int64', 'Tot Fwd Pkts': 'uint32', 'Tot Bwd Pkts': 'uint32', 'TotLen Fwd Pkts': 'uint32', 'TotLen Bwd Pkts': 'uint32', 'Fwd Pkt Len Max': 'uint16', 'Fwd Pkt Len Min': 'uint16', 'Fwd Pkt Len Mean': 'uint16', 'Fwd Pkt Len Std': 'uint16', 'Bwd Pkt Len Max': 'uint16', 'Bwd Pkt Len Min': 'uint16', 'Bwd Pkt Len Mean': 'uint16', 'Bwd Pkt Len Std': 'uint16', 'Flow Byts/s': 'float64', 'Flow Pkts/s': 'float64', 'Flow IAT Mean': 'int64', 'Flow IAT Std': 'uint64', 'Flow IAT Max': 'int64', 'Flow IAT Min': 'int64', 'Fwd IAT Tot': 'int64', 'Fwd IAT Mean': 'int64', 'Fwd IAT Std': 'uint64', 'Fwd IAT Max': 'int64', 'Fwd IAT Min': 'int64', 'Bwd IAT Tot': 'uint32', 'Bwd IAT Mean': 'uint32', 'Bwd IAT Std': 'uint32', 'Bwd IAT Max': 'uint32', 'Bwd IAT Min': 'uint32', 'Fwd PSH Flags': 'uint8', 'Bwd PSH Flags': 'uint8', 'Fwd URG Flags': 'uint8', 'Bwd URG Flags': 'uint8', 'Fwd Header Len': 'uint32', 'Bwd Header Len': 'uint32', 'Fwd Pkts/s': 'uint32', 

In [7]:
hasil

,Penggunaan memori lama (MB),Penggunaan memori baru (MB),Tipe Data Baru,Tipe Data Lama,Min,Max,Pengurangan Memori (MB)
Index,0.000128,NaN,NaN,NaN,NaN,NaN,NaN
Dst Port,129.863544,32.466014,uint16,int64,0.0,65535.0,97.397530
Protocol,129.863544,16.233071,uint8,int64,0.0,17.0,113.630473
Flow Duration,129.863544,129.863672,int64,int64,-919011000000.0,120000000.0,-0.000128
Tot Fwd Pkts,129.863544,64.931900,uint32,int64,1.0,309629.0,64.931644
Tot Bwd Pkts,129.863544,64.931900,uint32,int64,0.0,123118.0,64.931644
TotLen Fwd Pkts,129.863544,64.931900,uint32,int64,0.0,144391846.0,64.931644
TotLen Bwd Pkts,129.863544,64.931900,uint32,int64,0.0,156360426.0,64.931644
Fwd Pkt Len Max,129.863544,32.466014,uint16,int64,0.0,64440.0,97.397530
Fwd Pkt Len Min,129.863544,32.466014,uint16,int64,0.0,1460.0,97.397530


In [8]:
# dataframe dengan tipe data baru 
dataset_baru = pd.concat([data_1, data_2], axis=1)

# total penggunaan memori pada tipe data baru
dataset_baru.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16232943 entries, 0 to 16232942
Data columns (total 84 columns):
 #   Column             Dtype   
---  ------             -----   
 0   Dst Port           uint16  
 1   Protocol           uint8   
 2   Flow Duration      int64   
 3   Tot Fwd Pkts       uint32  
 4   Tot Bwd Pkts       uint32  
 5   TotLen Fwd Pkts    uint32  
 6   TotLen Bwd Pkts    uint32  
 7   Fwd Pkt Len Max    uint16  
 8   Fwd Pkt Len Min    uint16  
 9   Fwd Pkt Len Mean   uint16  
 10  Fwd Pkt Len Std    uint16  
 11  Bwd Pkt Len Max    uint16  
 12  Bwd Pkt Len Min    uint16  
 13  Bwd Pkt Len Mean   uint16  
 14  Bwd Pkt Len Std    uint16  
 15  Flow Byts/s        float64 
 16  Flow Pkts/s        float64 
 17  Flow IAT Mean      int64   
 18  Flow IAT Std       uint64  
 19  Flow IAT Max       int64   
 20  Flow IAT Min       int64   
 21  Fwd IAT Tot        int64   
 22  Fwd IAT Mean       int64   
 23  Fwd IAT Std        uint64  
 24  Fwd IAT Max        int

In [ ]:
# simpan hasil
hasil.to_csv("G:\\Universitas Indonesia\\Rancangan Paper\\Dataset CSE-CIC-IDS2018-AWS\\CIC2018\\hasil_datatype.txt", index=False)